In [1]:
import brainpy as bp
import brainpy.math as bm

class MyModel(bp.DynamicalSystem):

    def update(self, inp):
        # Define single-step dynamics update
        pass

In [2]:
def __init__(self):
    super().__init__()
    # State variable: neuron membrane potential
    self.u = bm.Variable(bm.zeros(self.num))
    # State variable: neuron firing rate
    self.r = bm.Variable(bm.zeros(self.num))
    # External input state
    self.inp = bm.Variable(bm.zeros(self.num))

In [3]:
import brainpy.math as bm  # :cite:p:`wang2023brainpy`

# Set simulation time step (unit: milliseconds)
bm.set_dt(0.1)

# Get current time step in the model
dt = bm.get_dt()

In [ ]:
class CANN1D(BaseCANN1D):
    def __init__(
        self,
        num: int,           # Number of neurons
        tau: float = 1.0,   # Time constant
        k: float = 8.1,     # Global inhibition strength
        a: float = 0.5,     # Connection width
        A: float = 10,      # External input amplitude
        J0: float = 4.0,    # Synaptic connection strength
        z_min: float = -π,  # Feature space minimum
        z_max: float = π,   # Feature space maximum
        **kwargs,
    ):
        ...

In [ ]:
def make_conn(self):
    # Calculate distances between all neuron pairs
    x_left = bm.reshape(self.x, (-1, 1))
    x_right = bm.repeat(self.x.reshape((1, -1)), len(self.x), axis=0)
    d = self.dist(x_left - x_right)

    # Compute connection strength using Gaussian function
    return (
        self.J0
        * bm.exp(-0.5 * bm.square(d / self.a))
        / (bm.sqrt(2 * bm.pi) * self.a)
    )

In [ ]:
def get_stimulus_by_pos(self, pos):
    return self.A * bm.exp(
        -0.25 * bm.square(self.dist(self.x - pos) / self.a)
    )

In [ ]:
def update(self, inp):
    self.inp.value = inp

    # Compute firing rate (divisive normalization)
    r1 = bm.square(self.u.value)
    r2 = 1.0 + self.k * bm.sum(r1)
    self.r.value = r1 / r2

    # Compute recurrent input
    Irec = bm.dot(self.conn_mat, self.r.value)

    # Update membrane potential using Euler method
    self.u.value += (
        (-self.u.value + Irec + self.inp.value)
        / self.tau * bm.get_dt()
    )

In [4]:
import brainpy.math as bm
from canns.models.basic import CANN1D

# Step 1: Set time step
bm.set_dt(0.1)

# Step 2: Create model instance
model = CANN1D(
    num=256,      # 256 neurons
    tau=1.0,      # Time constant
    k=8.1,        # Global inhibition
    a=0.5,        # Connection width
    A=10,         # Input amplitude
    J0=4.0,       # Connection strength
)

# Step 3: View model information
print(f"Number of neurons: {model.shape}")
print(f"Feature space range: [{model.z_min}, {model.z_max}]")
print(f"Connection matrix shape: {model.conn_mat.shape}")

Number of neurons: (256,)
Feature space range: [-3.141592653589793, 3.141592653589793]
Connection matrix shape: (256, 256)


In [5]:
# Generate external stimulus at pos=0
pos = 0.0
stimulus = model.get_stimulus_by_pos(pos)

# Run two step update
model(stimulus)	# or you can explicitly call model.update(stimulus)
model(stimulus)

# View current state
print(f"Firing rate shape: {model.r.value.shape}")
print(f"Max firing rate: {bm.max(model.r.value):.4f}")
print(f"Max membrane potential: {bm.max(model.u.value):.4f}")

Firing rate shape: (256,)
Max firing rate: 0.0024
Max membrane potential: 1.9275


In [6]:
import brainpy.math as bm  # :cite:p:`wang2023brainpy`
from canns.models.basic import CANN1D

# Setup environment
bm.set_dt(0.1)

# Create model (auto-initializes)
model = CANN1D(num=256, tau=1.0, k=8.1, a=0.5, A=10, J0=4.0)

# Print basic model information
print("=" * 50)
print("CANN1D Model Information")
print("=" * 50)
print(f"Number of neurons: {model.shape}")
print(f"Time constant tau: {model.tau}")
print(f"Global inhibition k: {model.k}")
print(f"Connection width a: {model.a}")
print(f"Input amplitude A: {model.A}")
print(f"Connection strength J0: {model.J0}")
print(f"Feature space: [{model.z_min:.2f}, {model.z_max:.2f}]")
print(f"Neural density rho: {model.rho:.2f}")

# Test stimulus generation
pos = 0.5
stimulus = model.get_stimulus_by_pos(pos)
print(f"\nStimulus position: {pos}")
print(f"Stimulus shape: {stimulus.shape}")
print(f"Max stimulus value: {bm.max(stimulus):.4f}")

# Run several update steps
print("\nRunning 100 update steps...")
for _ in range(100):
    model(stimulus)

print(f"Max firing rate: {bm.max(model.r.value):.6f}")
print(f"Max membrane potential: {bm.max(model.u.value):.6f}")

CANN1D Model Information
Number of neurons: (256,)
Time constant tau: 1.0
Global inhibition k: 8.1
Connection width a: 0.5
Input amplitude A: 10
Connection strength J0: 4.0
Feature space: [-3.14, 3.14]
Neural density rho: 40.74

Stimulus position: 0.5
Stimulus shape: (256,)
Max stimulus value: 9.9997

Running 100 update steps...
Max firing rate: 0.002427
Max membrane potential: 10.278063
